In [1]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [968 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,978 kB]
Get:13 http://archive.ubuntu.c

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Pet_Adoption").getOrCreate()

In [3]:
# Reading the csv file
df = spark.read.csv('/content/pet_adoption_renamed.csv',inferSchema=True, header=True)
df.show()

+------+--------+----------------+-------------+------+------+------------+----------+----------------+---------------+------------+--------------+-------------------+
|Pet_ID|Pet_Type|           Breed|Age_in_Months| Color|  Size|Weight_in_Kg|Vaccinated|Health_Condition|Days_in_Shelter|Adoption_Fee|Previous_Owner|Adoption_Likelihood|
+------+--------+----------------+-------------+------+------+------------+----------+----------------+---------------+------------+--------------+-------------------+
|   500|    Bird|        Parakeet|          131|Orange| Large| 5.039767823|         1|               0|             27|         140|             0|                  0|
|   501|  Rabbit|          Rabbit|           73| White| Large| 16.08672685|         0|               0|              8|         235|             0|                  0|
|   502|     Dog|Golden Retriever|          136|Orange|Medium| 2.076286279|         0|               0|             85|         385|             0|             

In [4]:
# Listing all the columns
df.columns

['Pet_ID',
 'Pet_Type',
 'Breed',
 'Age_in_Months',
 'Color',
 'Size',
 'Weight_in_Kg',
 'Vaccinated',
 'Health_Condition',
 'Days_in_Shelter',
 'Adoption_Fee',
 'Previous_Owner',
 'Adoption_Likelihood']

In [7]:
# Description of the age, days in shelter and adoption fee column
df.select(["Age_in_Months", "Days_in_Shelter", "Adoption_Fee"]).describe().show()

+-------+-----------------+------------------+------------------+
|summary|    Age_in_Months|   Days_in_Shelter|      Adoption_Fee|
+-------+-----------------+------------------+------------------+
|  count|             2007|              2007|              2007|
|   mean|92.27952167414051|43.974090682610864|249.14200298953662|
| stddev|52.14836276388298|25.740253386951636|142.88703981164835|
|    min|                1|                 1|                 0|
|    max|              179|                89|               499|
+-------+-----------------+------------------+------------------+



In [8]:
# Schema
df.printSchema()

root
 |-- Pet_ID: integer (nullable = true)
 |-- Pet_Type: string (nullable = true)
 |-- Breed: string (nullable = true)
 |-- Age_in_Months: integer (nullable = true)
 |-- Color: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Weight_in_Kg: double (nullable = true)
 |-- Vaccinated: integer (nullable = true)
 |-- Health_Condition: integer (nullable = true)
 |-- Days_in_Shelter: integer (nullable = true)
 |-- Adoption_Fee: integer (nullable = true)
 |-- Previous_Owner: integer (nullable = true)
 |-- Adoption_Likelihood: integer (nullable = true)



In [63]:
# Likelihood of each pet type to be adopted
count_df = df.groupBy("Pet_Type").sum()
count_df.select("Pet_Type", "sum(Adoption_Likelihood)").show()

+--------+------------------------+
|Pet_Type|sum(Adoption_Likelihood)|
+--------+------------------------+
|    Bird|                     147|
|     Cat|                     145|
|     Dog|                     242|
|  Rabbit|                     125|
+--------+------------------------+



In [90]:
# Maximum price is pet type is likely to be adopted
avg_df = df.groupBy("Pet_Type").max()
avg_df.select("Pet_Type", "max(Adoption_Fee)").show()

+--------+-----------------+
|Pet_Type|max(Adoption_Fee)|
+--------+-----------------+
|    Bird|              499|
|     Cat|              499|
|     Dog|              498|
|  Rabbit|              499|
+--------+-----------------+



In [85]:
# Number of different sizes for all pets
groups_size = df.groupBy('Size')
sizing =groups_size.agg({'Size':'count'})
sizing.show()

+------+-----------+
|  Size|count(Size)|
+------+-----------+
|Medium|        714|
| Small|        663|
| Large|        630|
+------+-----------+



In [86]:
import pandas as pd
pandas_df = sizing.toPandas()
pandas_df.head()

,Size,count(Size)
0,Medium,714
1,Small,663
2,Large,630


In [88]:
 # Install plotly
!pip install -q plotly

fig = px.bar(x=pandas_df['Size'], y=pandas_df['count(Size)'])
fig.update_layout(xaxis_title='Size', yaxis_title='# of Pet Sizes')
fig.show()


In [35]:
# Average adoption fee per breed
groups = df.groupBy('breed')
avg_price_per_breed =groups.agg({'Adoption_Fee':'Avg'})
avg_price_per_breed.show()

+----------------+------------------+
|           breed| avg(Adoption_Fee)|
+----------------+------------------+
|         Persian|247.30555555555554|
|        Labrador|248.57512953367876|
|        Parakeet|261.79671457905545|
|          Poodle|244.07784431137725|
|Golden Retriever|249.82716049382717|
|         Siamese|247.22134387351778|
|          Rabbit|240.27789046653143|
+----------------+------------------+



In [36]:
import pandas as pd
pandas_df = avg_price_per_breed.toPandas()
pandas_df.head()

,breed,avg(Adoption_Fee)
0,Persian,247.305556
1,Labrador,248.575130
2,Parakeet,261.796715
3,Poodle,244.077844
4,Golden Retriever,249.827160


In [38]:
import plotly.express as px
# Create a Plotly graph.

fig = px.line(pandas_df, x="breed", y="avg(Adoption_Fee)", title='Average Adoption Fee per Breed')
fig.show()



The average adoption fee for the Parakeet was the highest, while buying a rabbit was the cheapest

In [39]:
# Create a temporary view
df.createOrReplaceTempView('pet_adoption')

In [40]:
spark.sql("select * from pet_adoption limit 10").show()

+------+--------+----------------+-------------+------+------+------------+----------+----------------+---------------+------------+--------------+-------------------+
|Pet_ID|Pet_Type|           Breed|Age_in_Months| Color|  Size|Weight_in_Kg|Vaccinated|Health_Condition|Days_in_Shelter|Adoption_Fee|Previous_Owner|Adoption_Likelihood|
+------+--------+----------------+-------------+------+------+------------+----------+----------------+---------------+------------+--------------+-------------------+
|   500|    Bird|        Parakeet|          131|Orange| Large| 5.039767823|         1|               0|             27|         140|             0|                  0|
|   501|  Rabbit|          Rabbit|           73| White| Large| 16.08672685|         0|               0|              8|         235|             0|                  0|
|   502|     Dog|Golden Retriever|          136|Orange|Medium| 2.076286279|         0|               0|             85|         385|             0|             

In [44]:
# Querying all the pet types that are most likely to be adopted by the cheapest price.
query = """
(SELECT * FROM pet_adoption
WHERE Adoption_Likelihood==1
ORDER BY Adoption_Fee ASC)
"""
spark.sql(query).show()

+------+--------+--------+-------------+------+------+------------+----------+----------------+---------------+------------+--------------+-------------------+
|Pet_ID|Pet_Type|   Breed|Age_in_Months| Color|  Size|Weight_in_Kg|Vaccinated|Health_Condition|Days_in_Shelter|Adoption_Fee|Previous_Owner|Adoption_Likelihood|
+------+--------+--------+-------------+------+------+------------+----------+----------------+---------------+------------+--------------+-------------------+
|  1512|    Bird|Parakeet|          139|  Gray|Medium| 25.64543553|         1|               0|             28|           0|             0|                  1|
|  2156|     Dog|Labrador|           52|Orange| Large| 27.86211889|         1|               0|             29|           0|             0|                  1|
|  2173|    Bird|Parakeet|           46| White|Medium| 26.60066929|         1|               0|              5|           0|             1|                  1|
|  1420|     Dog|Labrador|           45|

In [91]:
# The total number of different pet types in the dataset
spark.sql("""
  SELECT
    pet_type,
    count(*) AS number_of_pet_types
  FROM pet_adoption
  GROUP BY pet_type
  ORDER BY 2 DESC
  """).show()


+--------+-------------------+
|pet_type|number_of_pet_types|
+--------+-------------------+
|     Dog|                522|
|     Cat|                505|
|  Rabbit|                493|
|    Bird|                487|
+--------+-------------------+



In [93]:
# Top 3 pets that are most likely to be adopted of each pet and their breed
spark.sql("""
SELECT
  Pet_Type,
  breed,
  count(*) breedCount
FROM pet_adoption
WHERE Adoption_Likelihood ='1' GROUP BY Pet_Type, breed
ORDER BY breedCount desc
LIMIT 3
""").show(truncate=False)

+--------+--------+----------+
|Pet_Type|breed   |breedCount|
+--------+--------+----------+
|Bird    |Parakeet|147       |
|Dog     |Labrador|139       |
|Rabbit  |Rabbit  |125       |
+--------+--------+----------+



In [100]:
# The top 5 youngest pet types to be me most likely adopted
spark.sql("""SELECT pet_type,breed,
   Age_in_Months,
   breed,
   Adoption_Fee
   FROM pet_adoption
   WHERE Age_in_Months < 90 AND Adoption_Likelihood ='1'
   ORDER BY Age_in_Months asc
   LIMIT 5
   """).show()

+--------+----------------+-------------+----------------+------------+
|pet_type|           breed|Age_in_Months|           breed|Adoption_Fee|
+--------+----------------+-------------+----------------+------------+
|     Dog|Golden Retriever|            1|Golden Retriever|          87|
|    Bird|        Parakeet|            1|        Parakeet|         255|
|     Dog|Golden Retriever|            1|Golden Retriever|          77|
|     Dog|Golden Retriever|            1|Golden Retriever|         146|
|    Bird|        Parakeet|            1|        Parakeet|         279|
+--------+----------------+-------------+----------------+------------+



In [99]:
# The top 5 oldest pet types to be me most likely adopted
spark.sql("""SELECT pet_type,breed,
   Age_in_Months,
   breed,
   Adoption_Fee
   FROM pet_adoption
   WHERE Age_in_Months > 90 AND Adoption_Likelihood ='1'
   ORDER BY Age_in_Months desc
   LIMIT 5
   """).show()

+--------+----------------+-------------+----------------+------------+
|pet_type|           breed|Age_in_Months|           breed|Adoption_Fee|
+--------+----------------+-------------+----------------+------------+
|    Bird|        Parakeet|          179|        Parakeet|         488|
|     Dog|Golden Retriever|          179|Golden Retriever|          70|
|  Rabbit|          Rabbit|          179|          Rabbit|         198|
|    Bird|        Parakeet|          179|        Parakeet|         197|
|     Dog|Golden Retriever|          178|Golden Retriever|         351|
+--------+----------------+-------------+----------------+------------+

